In this Notebook, we will convert the timeML files to .txt file and then ready the <Make Sentence> lines with the help of re, then map the eiid to eid and vice versa 

In [8]:
import os
import pandas as pd

def TML_to_TXT(data):
    path = "../dataset/tempeval-3-raw/" + data+ "/"
    out_dir = "D:/ML_Projects/RUG_Research_Training/modified_to_txt_dataset/"+data+"/"
    list_files = [i for i in os.listdir(path) if i.endswith(".tml")]
    # Loop through the tml files
    for file_ in list_files: 
        f1 = open(path+file_)
        f2 = open(out_dir+file_[:-4]+".txt", "w+")
        for lines in f1:
            f2.write(lines)
        f1.close()
        f2.close()


In [9]:
TML_to_TXT("AQUAINT")

In [10]:
TML_to_TXT("Platinum")

In [11]:
TML_to_TXT("TimeBank")

Here, we will save the eiid ans eid in a csv file
<MAKEINSTANCE eventID="e1" eiid="ei1" tense="PAST" aspect="NONE" polarity="POS" pos="UNKNOWN"/>

In [48]:
import os
import json
def convert_eiid_to_eid(data):
    out_dir = "../dataset/tempeval-3-eiid-to-eid/"+data+"/"
    path = "D:/ML_Projects/RUG_Research_Training/modified_to_txt_dataset/" + data+ "/"
    file_list = os.listdir(path)
    read_me = open(out_dir+"README.txt", "w+")
    for file_name in file_list:
        my_file = open(path+file_name, "r")
        eiid_2_eid = {}
        for lines in my_file.readlines():
            tokens = lines.split(' ')
            if tokens[0]=="<MAKEINSTANCE":
                for token in tokens: 
                    if token.startswith("eventID"):
                        eid= token
                    elif token.startswith("eiid"):
                        eiid= token
                eid = eid[9:-1]
                eiid = eiid[6:-1]
                if eiid in eiid_2_eid.keys():
                    read_me.write(f"for {file_name} eiid: {eiid} already exists for eid: {eiid_2_eid[eiid]} \n")
                else:
                    eiid_2_eid[eiid]=eid
        with open(out_dir+file_name[:-4]+".json", "w") as json_file:
            json.dump(eiid_2_eid, json_file)
    read_me.close()

In [49]:
convert_eiid_to_eid("AQUAINT")

In [50]:
convert_eiid_to_eid("Platinum")

In [51]:
convert_eiid_to_eid("TimeBank")

Read the matres files and create txt file in the required format
docid \tab sen1 \tab sen2 \tab s1:loc_1 \tab verb1 \tab s2:loc_2 \tab verb2 \tab relation 

In [1]:
import re
import json
import pandas as pd 

def extract_doc_info(path_col_file, path_eiid_file):
    col_file = open(path_col_file)
    eiid_2_eid = json.load(open(path_eiid_file))
    sentence_list = []
    eid_2_sen_idx = {}
    eid_2_sen_loc = {}
    prev_sen_idx = 0
    sentence = ""
    for lines in col_file.readlines():
        if len(lines)<2:
            continue
        col_tokens = lines.split('\t')
        if col_tokens[1].startswith("t"):
            token = col_tokens[0]
            sen_idx = col_tokens[2]
            sen_idx = int (sen_idx)
            if sen_idx != prev_sen_idx:
                if len(sentence)>0:
                    sentence_list.append(sentence) 
                sentence = token
            else: 
                sentence = sentence+ " " + token
            prev_sen_idx = sen_idx
            eid = col_tokens[4]
            if eid.startswith("e"):
                eid_2_sen_idx[eid]= sen_idx-1
                eid_2_sen_loc[eid] = len(sentence.split(' '))
    if len(sentence)>0:
        sentence_list.append(sentence) 
    return sentence_list, eiid_2_eid, eid_2_sen_idx, eid_2_sen_loc
    

def format_matres_dataset(data_dir):
    if data_dir == "AQUAINT":
        matres_f_name = "dev_aquaint.txt"
        col_dir_name = "TempEval3-train_COL/"
    elif data_dir == "Platinum":
        matres_f_name = "test_platinum.txt"
        col_dir_name = "TempEval3-eval_COL/"
    else:
        matres_f_name = "train_timebank.txt"
        col_dir_name = "TempEval3-train_COL/"
    path_matres="../dataset/matres/"+matres_f_name
    write_to = "../dataset/matres/relations/" + matres_f_name
    matres_file = open(path_matres)
    write_to_file = open(write_to, "w+")
    path_col_file = "../dataset/tempeval-3-col/"+ col_dir_name
    path_eiid_file = '../dataset/tempeval-3-eiid-to-eid/'+data_dir+"/"
    prev_doc_id = ""
    for lines in matres_file.readlines():
        tokens = re.split('\t|\n', lines)
        docid= tokens[0]; verb1 = tokens[1]; verb2 = tokens[2]; eiid_v1 = tokens[3]; eiid_v2= tokens[4]; relation = tokens[5]
        if docid != prev_doc_id: 
            if docid=="nyt_20130321_sarcozy":
                sentence_list, eiid_2_eid, eid_2_sen_idx, eid_2_sen_loc = extract_doc_info(path_col_file+"nyt_20130321_sarkozy.col", path_eiid_file+"nyt_20130321_sarcozy.json")    
            else:
                sentence_list, eiid_2_eid, eid_2_sen_idx, eid_2_sen_loc = extract_doc_info(path_col_file+docid+".col", path_eiid_file+docid+".json")
            if eiid_v1=='0' and verb1=="think":
                print(docid)
                eiid_v1 = "1221"
            eid1 = eiid_2_eid["ei"+eiid_v1]
            eid2 = eiid_2_eid["ei"+eiid_v2]
            write_to_file.write(f"{docid}\t{sentence_list[eid_2_sen_idx[eid1]]}\t{sentence_list[ eid_2_sen_idx[eid2] ]}\ts1:{eid_2_sen_loc[eid1]-1}\t{verb1}\ts2:{eid_2_sen_loc[eid2]-1}\t{verb2}\t{relation}\n")
        else:
            write_to_file.write(f"{docid}\t{sentence_list[eid_2_sen_idx[eid1]]}\t{sentence_list[ eid_2_sen_idx[eid2] ]}\ts1:{eid_2_sen_loc[eid1]-1}\t{verb1}\ts2:{eid_2_sen_loc[eid2]-1}\t{verb2}\t{relation}\n")
    write_to_file.close()

In [2]:
format_matres_dataset("AQUAINT")

In [3]:
format_matres_dataset("Platinum")

In [4]:
format_matres_dataset("TimeBank")

WSJ910225-0066
WSJ910225-0066
WSJ910225-0066
WSJ910225-0066


In [17]:
import json
path_col_file = "../dataset/tempeval-3-col/"+ "TempEval3-eval_COL/AP_20130322.col"
path_eiid_file = '../dataset/tempeval-3-eiid-to-eid/'+"Platinum/AP_20130322.json"
path_matres="../dataset/matres/"+"test_platinum.txt"
col_file = open(path_col_file)
eiid_2_eid = json.load(open(path_eiid_file))
matres_file = open(path_matres)

In [31]:
x = matres_file.readline()

In [32]:
import re
re.split('\t|\n', x)

['WSJ_20130322_159', 'apologized', 'seemed', '1', '10', 'BEFORE', '']

In [7]:
type(eiid_2_eid)

dict

In [4]:
eiid_2_eid.keys()

dict_keys(['ei2', 'ei3', 'ei4', 'ei5', 'ei6', 'ei7', 'ei8', 'ei9', 'ei10', 'ei11', 'ei12', 'ei13', 'ei16', 'ei17', 'ei19', 'ei20', 'ei21', 'ei22', 'ei26', 'ei27', 'ei2001', 'ei28', 'ei29', 'ei30', 'ei31', 'ei1000028', 'ei2002', 'ei2003', 'ei2004', 'ei1000030', 'ei1000032', 'ei1000033', 'ei1000035'])

In [18]:
col_file = open(path_col_file)
for lines in col_file.readlines():
    print(lines.split('\t'))

['DCT_2013-03-22', 'O', 'O', 'O', 'O', 'O', 'O', 'tmx0', 'DATE', '2013-03-22', 'O', 'O', 'NN1', 'B-NP', 'dct_2013-03-22', 'LS', 'O', 'O\n']
['\n']
['The', 't1', '1', 'the', 'O', 'O', 'O+O+O', 'O', 'O', 'O', 'O', 'O', 'AT0', 'B-NP', 'the', 'DT', 'O', 'O\n']
['flu', 't2', '1', 'flu', 'O', 'O', 'O+O+O', 'O', 'O', 'O', 'O', 'O', 'NN1', 'I-NP', 'flu', 'NN', 'O', 'O\n']
['season', 't3', '1', 'season', 'e1000028', 'OCCURRENCE', 'NONE+NONE+POS', 'O', 'O', 'O', 'O', 'O', 'NN1', 'I-NP', 'season', 'NN', 't1:NMOD||t2:NMOD', 'O\n']
['is', 't4', '1', 'be', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'VBZ', 'B-VP', 'be', 'VBZ', 't3:SBJ||t5:VC||t7:P||t8:COORD||t21:P', 'mainVb\n']
['winding', 't5', '1', 'wind', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'VVG', 'I-VP', 'wind', 'VBG', 't6:PRT', 'mainVb\n']
['down', 't6', '1', 'down', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'AVP', 'B-PRT', 'down', 'RP', 'O', 'O\n']
[',', 't7', '1', ',', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PUN', 'O', ',', ',', 'O', 'O\n']
['